## cluster

    This notebook demonstrate:
    1.data preprocessing;
    2.Classical leiden clustering and louvain clustering
    3.STAGATE clustering method
    4.SPAGCN clustering method

### preprocessing--normalization (SCtransform)

> Since the original data and the post-SCT data are placed in one adata, the adata data will be too large, which will make subsequent analysis difficult.
> Therefore, here we choose to store the original data and the data after SCT separately.

- sct_adata_new: stores new data, including `gene expression data after SCT` and `new coordinate data after registration`.
- sct_adata_raw: stores raw data, including `raw gene expression` data and `raw coordinate data`.

In [ ]:
import os
import anndata as ad
import stDrosophila as sd

folder = "/media/yao/Elements SE/BGI_Paper/L3_new/L3_b/align_adata/L3_b_bin20"
files = [os.path.join(root, filename)for root, dirs, files in os.walk(folder)for filename in files]
files.sort()
for file in files:
    # input adata
    adata = ad.read(file)
    # run sd.tl.sct_stereo
    sct_adata_new, sct_adata_raw = sd.tl.sct_stereo(adata, filter_hvgs=True)
    # save
    sct_adata_new.write_h5ad(
        f"/media/yao/Elements SE/BGI_Paper/L3_new/L3_b/sct_adata/L3_b_bin20/{sct_adata_new.obs['slice'][0]}.h5ad")
    sct_adata_raw.write_h5ad(
        f"/media/yao/Elements SE/BGI_Paper/L3_new/L3_b/sct_adata/L3_b_bin20_raw/{sct_adata_raw.obs['slice'][0]}.h5ad")
    print(f"----{sct_adata_new.obs['slice'][0]} successfully!")

----L3_b_S25 successfully!
----L3_b_S26 successfully!
----L3_b_S27 successfully!
----L3_b_S28 successfully!
----L3_b_S29 successfully!
----L3_b_S30 successfully!


### STAGATE clustering

In [ ]:
import anndata as ad
import scanpy as sc
import stDrosophila as sd
import torch

# input adata
ipath = 'D:\BGIpy37_pytorch113\Data_Anno_H5ad\sct_analysis_anno_h5ad\L2_a_fatbody_pre.h5ad'
adata = ad.read(ipath)
# run sd.tl.STAGATE_SNN
adata = sd.tl.STAGATE_SNN(adata, type="3D", slice_col="slice", cutoff_2D=3, cutoff_Zaxis=3,
                          n_epoch=1000, lr=1e-4, weight_decay=1e-4, device=torch.device('cuda:0'), verbose=True)

In [ ]:
# cluster by leiden, louvain or mclust(R)
sc.pp.neighbors(adata, use_rep='STAGATE', n_neighbors=20)
sc.tl.umap(adata)
resolution = 0.5
sc.tl.louvain(adata, resolution=resolution, key_added=f"louvain_{str(resolution)}")
## sc.tl.leiden(adata, resolution=resolution, key_added=f"leiden_{str(resolution)}")
adata.write_h5ad(r"D:\BGIpy37_pytorch113\Data_Anno_H5ad\sct_analysis_anno_h5ad\L2_a_fatbody_STAGATE_louvain.h5ad")